In [1]:
import os
import PyPDF2 as pdf
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI  # <-- Corrected import

os.environ['OPENAI_API_KEY']=

In [2]:
def getTextFromPDF()->str:
    with open('Quiz.pdf',mode='rb') as contentFile:
        convertedText = ""
        pdf_reader = pdf.PdfReader(contentFile)
        for i in range(0,len(pdf_reader.pages)):
            page = pdf_reader.pages[i]
            pageText = page.extract_text()
            convertedText+=pageText
        file_path = "Loader.txt"
        with open(file_path, "w+") as file:
            file.write(convertedText)
            file.seek(0)
        return file_path

In [3]:
loader = TextLoader(getTextFromPDF())
documents = loader.load()
text_splitter  =  RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=50)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
store = Chroma.from_documents(texts,embeddings,collection_name="Quiz-PDF")

In [ ]:
llm = ChatOpenAI(temperature=0.8, model="gpt-3.5-turbo")
chain = RetrievalQA.from_chain_type(llm,retriever=store.as_retriever())

In [ ]:
prompt = "The document consists of questions and multiple choice options, return only the questions from the document in a bullet points"
response = chain.run(prompt)

In [ ]:
print(response)